In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200804093425-0000
KERNEL_ID = 211d9286-cb11-4639-8671-6209e08e70ec


In [2]:
!pip install watson-transformer

    100% |████████████████████████████████| 829kB 3.3MB/s eta 0:00:01
    100% |████████████████████████████████| 6.5MB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 378kB 4.5MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 3.8MB/s eta 0:00:01
    100% |████████████████████████████████| 225kB 3.2MB/s eta 0:00:01
    100% |████████████████████████████████| 552kB 4.2MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 3.1MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 4.8MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 204kB 5.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 2.6MB/s eta 0:00:01
    100% |████████████████████████████████| 163kB 3.7MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 5.1MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/f1/29/05/ab91fed96d22f69b37c4331f

In [3]:
from watson_transformer import WatsonServiceTransformer, JSONTransformer, FlatColumnTransformer
from watson_transformer.service import STT, NLU
from watson_transformer.contrib.stt import DefaultSTTParser
from watson_transformer.contrib.nlu import DefaultNLUParser
import watson_transformer.contrib.readers as readers
from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions, ConceptsOptions, SentimentOptions, EmotionOptions

import json
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [4]:
import pandas as pd
import time
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml import Pipeline

# Watson Transformer package - Job execution
###  Batch job execution for STT trancriptions
###  Red items need to be substitted for execution

## [Part 1: Define services and transformers](#pt1)

## [Part 2: Pipeline & Execution](#pt2)


<a id='pt1'></a> 
## Part 1: Define services and transformers

1. [Enter Service Credentials](#pt1_s1)
2. [Load file names to read](#pt1_s2)
    - [from Object Storage](#pt1_s2a)
    - [from project file](#pt1_s2b) 
3. [Create reader for IBM Cloud Object Storage](#pt1_s3)
4. [Define Watson Service & Initialize transformers](#pt1_s4)
   

<a id='pt1_s1'></a> 
###  <p style="color:Red">  1. Enter Service Credentials </p>

In [33]:
# Credentials to cloud object storage

api_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
auth_endpoint = 'https://iam.ng.bluemix.net/oidc/token'
private_endpoint_url='https://s3.us-south.objectstorage.service.networklayer.com' 
public_endpoint_url='https://s3.us-south.objectstorage.softlayer.net' 
instance_id = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
Bucket ='stt-samples'

In [34]:
# The code was removed by Watson Studio for sharing.

In [35]:
# The code was removed by Watson Studio for sharing.

In [36]:
#STT Credentials 

stt_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 
stt_endpoint = 'https://stream.watsonplatform.net/speech-to-text/api'

In [37]:
import boto3
import types
from botocore.client import Config
import ibm_boto3

cos_clstorage  = ibm_boto3.resource(service_name='s3',
                               ibm_api_key_id=api_key,
                               ibm_auth_endpoint=auth_endpoint,
                               config=Config(signature_version='oauth'),
                               endpoint_url=public_endpoint_url,
                               ibm_service_instance_id= instance_id)

<a id='pt1_s2'></a> 
### 2. Load Data

<a id='pt1_s2a'></a> 
### <p style="color:Red"> Option A) Load data file names from object storage (change file extension if necessary) </p>

In [52]:
#Function iterates over IBM COS and appends every object into a list, and then adds them to a dataframe to use later 
#enter extension to keep for files - audio_ext=None if you want to retrieve everything - 'mp3' for ending in .mp3, etc 

def get_bucket_contents(bucket_name,audio_ext=None):
    files_list=[]
    try: 
        files = cos_clstorage.Bucket(bucket_name).objects.all()
        for file in files:
            #print("Item: {0} ({1} bytes).".format(file.key, file.size))
            files_list.append(file.key)      
    
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))
    
    #dataframe with all the available files in the object storage (even call logs)
    bucket_files = pd.DataFrame({'audio_file': files_list})
    if audio_ext == None:
        audio_files = bucket_files
    else: 
        audio_files = bucket_files[bucket_files['audio_file'].str.contains(audio_ext)]
    return audio_files

In [53]:
audio_files = get_bucket_contents(Bucket,audio_ext='mp3') 

<a id='pt1_s2a'></a> 
### <p style="color:Red"> Option B) Load data file names from object storage (change file name if importing from project)</p>

In [30]:
mp3_project_files = pd.read_csv(project.get_file('MP3_AudioFiles.csv'), sep=',')
print(mp3_project_files.count())
mp3_project_files.head()

audio_file    20
dtype: int64


,audio_file
0,incomingcall_sample1.mp3
1,incomingcall_sample2.mp3
2,incomingcall_sample3 copy 2.mp3
3,incomingcall_sample3 copy 3.mp3
4,incomingcall_sample3 copy.mp3


<a id='pt1_s3'></a> 
### 3. Create reader for IBM COS

In [25]:
import types
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0

"""
"
" IBM COS reader
"
"""
        
def ibm_cos_reader(audio_file, maximum_size=200*1024*1024):
    # boto3 is not thread safe
    # issue: https://github.com/boto/botocore/issues/1246
    # ref: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/core/session.html
    # create new session for each thread, boto3 is not thread safe
    session = ibm_boto3.session.Session() 
    cos_client = session.client(service_name='s3',
                                ibm_api_key_id=api_key,
                                ibm_auth_endpoint=auth_endpoint,
                                config=Config(signature_version='oauth'),
                                endpoint_url=public_endpoint_url)
    # read file object
    file_obj = cos_client.get_object(Bucket=Bucket, Key=audio_file)
    # make sure file size is less than maximum size
    if int(file_obj['ContentLength']) >= maximum_size:
        return None
    else:
        audio_stream = file_obj['Body']
        if not hasattr(audio_stream, "__iter__"): audio_stream.__iter__ = types.MethodType( __iter__, audio_stream)
        return audio_stream

<a id='pt1_s4'></a> 
### <p style="color:Red"> 4. Define Watson Service & Initialize Transformers: this is where you are defining number of worker nodes </p>

In [26]:
#defining STT service
stt = STT(token = stt_token,
                   endpoint = stt_endpoint,
                   reader = ibm_cos_reader,
                   model='en-US_ShortForm_NarrowbandModel',
                   inactivity_timeout=-1,
                   profanity_filter=False,
                   max_alternatives=1,
                   split_transcript_at_phrase_end=False,
                   content_type='audio/mp3')

In [124]:
wt = WatsonServiceTransformer(inputCol='audio_file', 
                               outputCol='stt_response',
                               vectorization=True,
                               max_workers =20,
                               service=stt)

In [32]:
jt_stt = JSONTransformer(inputCol='stt_response',
                     outputCol='transcript',
                     removeInputCol=False,
                     parser = DefaultSTTParser())


<a id='pt2'></a> 

## Part 2: Pipeline & Execution
1. [Create PySpark DataFrame and pipeline](#pt2_s1)
2. [Define needed variables](#pt2_s2)
3. [Logger](#pt2_s3)
4. [Batch Runner](#pt2_s4)
5. [Launch Pad](#pt2_5)

<a id='pt2_s1'></a> 
### <p style="color:Red"> 1.  Create PySpark DataFrame and pipeline:</p> 
    substitute "mp4_project_files" for dataframe name containing the audio files


In [119]:
foo = spark.createDataFrame(mp3_project_files).repartition(5)
pipeline_stt = Pipeline(stages=[wt, jt_stt])

In [122]:
fp = pipeline_stt.fit(foo)
fp.transform(foo).explain()

== Physical Plan ==
*(2) Project [audio_file#119, stt_response#122, pythonUDF0#131 AS transcript#126]
+- BatchEvalPython [DefaultSTTParser(stt_response#122)], [audio_file#119, stt_response#122, pythonUDF0#131]
   +- *(1) Project [audio_file#119, pythonUDF0#130 AS stt_response#122]
      +- ArrowEvalPython [vectorized_udf(audio_file#119)], [audio_file#119, pythonUDF0#130]
         +- Exchange RoundRobinPartitioning(5)
            +- Scan ExistingRDD[audio_file#119]


<a id='pt2_s2'></a> 
### <p style="color:Red">  2. Define Variables (choose your own variables) </p> 

In [114]:
df = mp3_project_files #pandas dataframe containing list of file names 
batch_size=5000 #number of files to process in each batch if there are too many files
num_partitions=250 #number of partitions per batch (partititions = batch_size/max_workers)
log_csv_name='company_pipeline_log1.csv' #it will only create one log per job - remember to include .csv 
file_prefix='company_transcription_part_2'

<a id='pt2_s3'></a> 
## 3. Logger

In [115]:
import sys
import logging
from datetime import datetime
logger = logging.getLogger(__name__)

In [116]:
"""
"
" write log to file
"
"""
def project_logger(filename, info, new_log_file=False):
    timestamp = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    
    if new_log_file:
        try:
            # create empty log file
            foo = pd.DataFrame({"timestamp":[timestamp], "log":[info]})
            project.save_data(filename, foo.to_csv(index=False, sep='|'), overwrite=True)
        except:
            pass
    else:  
        try:
            df = pd.read_csv(project.get_file(filename), sep='|')
            df.loc[df.shape[0]] = [timestamp, info]
            project.save_data(filename, df.to_csv(index=False, sep='|'), overwrite=True)
        except:
            pass
        

<a id='pt2_s4'></a> 
## 4. Batch Runner

In [117]:
"""
"
" functiom provide another layer of fault resistance on top of Spark engine.
"
"""
def batch_runner(df, pipeline, project, 
                               batch_start=1, 
                               batch_size=batch_size, 
                               num_partitions=num_partitions,
                               file_prefix=None,
                               log_csv_name=log_csv_name,
                               overwrite=False,
                               verbose=True):
    """
    @param::df: the input pandas dataframe
    @param::pipeline: the fitted Spark ML pipeline
    @param::project: the Watson project instance
    @param::batch_start: the row pointer to where the batch process begin
    @param::batch_size: the size of the data batch
    @param::num_partitions: the number of partitions
    @param::file_prefix: the prefix of the output data file
    @param::overwrite: whether or not overwrite the output data file when it already exist
    @param::verbose: toggle log info
    return: None
    """
    if file_prefix == None:
        file_prefix = str(uuid.uuid4())
    
    row_count = df.shape[0]
    i = batch_start - 1
    iteration = 1
    not_finished = True
    
    while not_finished:
        batch_df = df.iloc[i:i+batch_size]
        if i+batch_size >= row_count:
            not_finished = False  
        try:
            sdf = spark.createDataFrame(batch_df).repartition(num_partitions)
            start = time.perf_counter()
            tmp = pipeline.transform(sdf).toPandas()
            delta = time.perf_counter() - start
            filename = '%s_result_batch_%s.csv'%(file_prefix, iteration)
            project.save_data(filename, tmp.to_csv(index=False, sep='|'), overwrite=overwrite)
            if verbose:
                print('> batch %s of %s rows finished process in %d seconds.'%(iteration, batch_df.shape[0], delta))
                project_logger(log_csv_name, 'Batch %s of %s rows finished process in %d seconds.'%(iteration, batch_df.shape[0], delta))
                
        except Exception as e:
            error_filename = '%s_failed_batch_%s.csv'%(file_prefix, iteration)
            project.save_data(error_filename, batch_df.to_csv(index=False, sep='|'), overwrite=overwrite)
            print('> batch %s failed: %s'%(iteration, e))
            project_logger(log_csv_name, 'Batch %s failed: %s'%(iteration, e)) 
        finally:
            i += batch_size
            iteration += 1

<a id='pt2_5'></a> 
## 5. Launch pad

In [118]:
"""
"
" launch pad
"
"""


print('> the input size of dataframe: ', df.shape)
project_logger(log_csv_name, "*** Pipeline is starting ....", new_log_file=True)  
project_logger(log_csv_name, "The input size of dataframe: (%s, %s)"%(df.shape))  
df.head(3)


# execute the batch runner
print('> the batch runner start executing...')
project_logger(log_csv_name, "The batch runner start executing...") 

batch_runner(df, fp, project, batch_size=batch_size, num_partitions=num_partitions, file_prefix=file_prefix, overwrite=True)

> the input size of dataframe:  (20, 1)
> the batch runner start executing...
> batch 1 of 20 rows finished process in 7 seconds.
